In [15]:
from pymongo import ASCENDING
from services.db_service import DbService
from datetime import datetime

In [16]:
tournaments = DbService().events_collection()

In [17]:
colors = {
    "W": "01-white.txt",
    "U": "02-blue.txt",
    "B": "03-black.txt",
    "R": "04-red.txt",
    "G": "05-green.txt",
    "M": "06-multicolored.txt",
    "C": "07-colorless.txt",
    "L": "08-lands.txt",
}

In [18]:
def get_pipeline(color):
    return [
        {"$match": {"date": {"$gte": datetime.strptime(
            "2024-04-01", "%Y-%m-%d"), "$lt": datetime.strptime("2024-07-01", "%Y-%m-%d")}}},
        {"$project": {"decklists": "$players.decklist"}},
        {"$unwind": "$decklists"},
        {"$group": {"_id": "$decklists"}},
        {"$lookup": {"from": "decklists", "localField": "_id",
                     "foreignField": "_id", "as": "decklists"}},
        {"$unwind": "$decklists"},
        {"$match": {"decklists.banned": {"$ne": True}}},
        {"$project": {"_id": "$decklists._id", "cards": {
            "$concatArrays": ["$decklists.maindeck", "$decklists.sideboard"]}}},
        {"$unwind": "$cards"},
        {"$group": {"_id": {"_id": "$_id", "name": "$cards"}, "count": {"$sum": 1}}},
        {"$project": {"_id": "$_id._id", "name": "$_id.name", "count": 1}},
        {"$group": {"_id": "$name", "count": {
            "$max": "$count"}, "total_decks": {"$sum": 1}}},
        {"$match": {"total_decks": {"$gte": 9}}},
        {"$lookup": {"from": "pool", "localField": "_id",
                     "foreignField": "name", "as": "pool"}},
        {"$unwind": "$pool"},
        {"$project": {
            "_id": "$pool._id", color: "$pool.color", "name": "$pool.name", "scryfall_name": "$pool.scryfall_name",
            "count": 1, "total_decks": 1
        }},
        {"$match": {color: color}},
        {"$sort": {"name": 1}},
    ]

In [19]:
for color, file in colors.items():    
    cards = list(tournaments.aggregate(get_pipeline(color)))
    with open(f"../pool/{file}", "w") as f:
        for card in cards:
            f.write(f"{card['count']} {card['scryfall_name']}\n")